# Test Base Encoder

In [1]:
from BaseEncoder import BaseEncoder
from transformers import AutoTokenizer, TFAutoModel

In [2]:
sentence = 'Aku seorang kapiten'

In [3]:
base_encoder = BaseEncoder()

In [4]:
sample_tokenized = base_encoder.tokenize(['Aku', 'seorang', 'kapiten'])

In [5]:
sample_tokenized.shape

torch.Size([40])

# Data Loader

In [6]:
from DataLoader import read_examples_from_file, ReviewDataset

FILE_PATH = "dataset/test_2k_output_Feb_26_2020_output_Feb_18_2021.tsv"

In [7]:
test_data = ReviewDataset(FILE_PATH)

In [8]:
len(test_data)

200

In [9]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)

# Test Model

In [10]:
from SpanMltri import SpanMltri
import torch
from torch import nn

Using cuda device


In [11]:
span_mltri = SpanMltri()

In [47]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(span_mltri.parameters(), lr=0.005)

BATCH_SIZE = 32
size = len(test_dataloader.dataset)
for batch, X in enumerate(test_dataloader):
    current_te_label_list = test_data.te_label_list[(batch)*BATCH_SIZE:(batch+1)*BATCH_SIZE]
    current_relations = test_data.relations[(batch-1)*BATCH_SIZE:batch*BATCH_SIZE]
    
    y_te_pred, span_ranges = span_mltri(X)
    
    y_te_true = []
    CURRENT_BATCH_SIZE = min(len(current_te_label_list), BATCH_SIZE)
    for i in range(CURRENT_BATCH_SIZE):
        y_ = []
        for span_range in span_ranges:
            if span_range in current_te_label_list[i]:
                label = current_te_label_list[i][span_range]
                if label == 'ASPECT':
                    y_.append(0)
                elif label == 'SENTIMENT':
                    y_.append(1)
            else: # label is O
                y_.append(2)        
        y_te_true.append(torch.Tensor(y_))
    y_te_true = torch.stack(y_te_true)
    y_te_true = y_te_true.to(torch.long)
    
    y_te_pred = y_te_pred.reshape(y_te_pred.shape[0]*y_te_pred.shape[1], y_te_pred.shape[-1])
    y_te_true = y_te_true.reshape(-1)
    loss = loss_fn(y_te_pred, y_te_true)
    
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 2 == 0:
        loss, current = loss.item(), batch * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

loss: 1.503387  [    0/  200]
loss: 0.569164  [   64/  200]
loss: 0.567262  [  128/  200]
loss: 0.569415  [   48/  200]


In [46]:
loss

tensor(1.5202, grad_fn=<NllLossBackward>)

## Train Term Scorer

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(span_mltri.parameters(), lr=0.005)

In [ ]:
BATCH_SIZE = 32

def train(dataloader, relations, te_labels, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, X in enumerate(dataloader):
        X = X.to(device)
        
        # Compute prediction error
        pred = model(X)
        
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [19]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()

In [20]:
input

tensor([[ 0.0212,  0.8463,  1.1895,  0.6229,  0.5368],
        [-1.4045,  0.8464,  0.2055,  1.6715,  0.7902],
        [ 1.5953, -0.3220,  0.1930, -1.0073,  0.1035]], requires_grad=True)

In [32]:
t.reshape(-1).shape

torch.Size([120])